<a href="https://colab.research.google.com/github/evilsizord/mscs-data-mining-hw2/blob/main/Data_Mining_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Naive Bayes Classifier
Daniel Evilsizor \
November 13, 2022

In [1]:
# Get Data from Kaggle.com

# NOTE: Requires you to have a Kaggle.com account. From your account you can generate an API key.
# It will be provided in kaggle.json. Upload the JSON file to this project BEFORE RUNNING.

# src: https://www.kaggle.com/general/74235

! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download gaveshjain/ford-sentence-classifiaction-dataset   # yes there is a typo in the actual URL
! mkdir sentence-dataset
! unzip ford-sentence-classifiaction-dataset.zip -d sentence-dataset

  0% 0.00/2.92M [00:00<?, ?B/s]
100% 2.92M/2.92M [00:00<00:00, 126MB/s]
Archive:  ford-sentence-classifiaction-dataset.zip
  inflating: sentence-dataset/sample_submission.csv  
  inflating: sentence-dataset/test_data.csv  
  inflating: sentence-dataset/train_data.csv  


In [2]:
# load all necessary libraries

import os
import pandas
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#from numpy import nan
import numpy as np

# Load and preview the data
input_train = pandas.read_csv('sentence-dataset/train_data.csv')
input_test = pandas.read_csv('sentence-dataset/test_data.csv')

# throw out the integer indexs before merging
input_train = input_train.drop(input_train.columns[[0]], axis=1)
input_test = input_test.drop(input_test.columns[[0]], axis=1)

all_data = pandas.concat([input_train, input_test])
# how many rows?
print(len(all_data), 'total rows loaded')


# throw out null sentences (see https://stackoverflow.com/a/56708633)
# todo: sigh why this not working..
#all_data = all_data[ all_data['New_Sentence'].str.strip().astype(bool) & all_data['New_Sentence'] != nan ]
#print(len(all_data), 'rows after removing empty')

all_data['New_Sentence'].replace('', np.nan, inplace=True)
all_data.dropna(subset=['New_Sentence'], inplace=True)
print(len(all_data), 'rows after removing empty 2')

#debug: what are values for type?
#print(all_data['Type'].unique())
categories = ['Responsibility', 'Requirement', 'Skill', 'SoftSkill', 'Education', 'Experience']

testdata = all_data[ all_data['Type'].isnull() ]
traindevdata = all_data[ all_data['Type'].notnull() ]

split = int(.5*len(traindevdata))
traindata = traindevdata[:split]
devdata = traindevdata[(split+1):]

print(len(traindata), 'training,', len(devdata), 'dev,', len(testdata), 'test rows loaded')

# preview the result
all_data.head(5)

75144 total rows loaded
73750 rows after removing empty 2
29501 training, 29500 dev, 14748 test rows loaded


,Sentence_id,New_Sentence,Type
0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility
1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility
2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement
3,GERSKL27235,"Previous business experience, including but no...",Skill
4,HONSSK18415,Delivering fast and right the first time.,SoftSkill


In [5]:
# get vocabulary statistics
vocab = {}
category_counts = {}
ps = PorterStemmer()

# helper function
def dict_increment(mydict, mykey):
  if not mykey:
    return  # if key is false, ignore
  if mykey in mydict:
    mydict[mykey] += 1
  else:
    mydict[mykey] = 1

for index, row in all_data.iterrows():
  if 'New_Sentence' in row:
    tokens = word_tokenize(row['New_Sentence'])

    # todo: remove stopwords?
    # todo: do we need to lowercase the tokens?
    stemwords = [ps.stem(tok) for tok in tokens]

    # remove punctuation
    # ref: https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer
    stemwords = list(filter(lambda token: token not in '.,', tokens))

    # convert to a set so we only get unique words per row (no duplicates)
    stemwords = set(stemwords)

    # get category (if train/dev data)
    myclass = row['Type'] if row['Type'] else False

    # increment overall category counts
    dict_increment(category_counts, myclass)

    # count overall word frequency, and frequency per class (label)
    for word in stemwords:
      if word in vocab:
        dict_increment(vocab[word], 'total_frequency')
        dict_increment(vocab[word], myclass)
      else:
        vocab[word] = {'total_frequency': 1}
        for cat in categories:
          vocab[word][cat] = 0
        dict_increment(vocab[word], myclass)
  else:
    print('ERROR: no sentence in row', row)

# preview
print('Found', len(vocab.keys()), 'words in vocabulary')

Found 37781 words in vocabulary


In [7]:
# Calculate probability of occurrance of each word
vocab_probs = {}
for word in vocab:
  vocab_probs[word] = vocab[word]['total_frequency'] / len(all_data)

# preview: words with highest probability
print('Top 5')
for w in sorted(vocab_probs, key=vocab_probs.get, reverse=True)[:5]:
  print(w, vocab_probs[w])

# preview: words with lowest probability
print('Bottom 5')
for w in sorted(vocab_probs, key=vocab_probs.get)[:5]:
  print(w, vocab_probs[w])

# Calculate	Conditional probability of all words based on category
#	P[“the” | Skill]  = # of skill documents containing “the” / num of all skill documents
# (we can only do this for the dev and train datasets, since we do not have categories given for the test data)
conditional_probs = {}
for cat in categories:
  for word in vocab:
    #x = sum(vocab[word][cat] for word in vocab)
    conditional_probs[word + '_' + cat] = vocab[word][cat] / category_counts[cat]

# preview
print('Top 5 Conditional probability (Responsibility)')
for w in sorted(conditional_probs, key=conditional_probs.get, reverse=True)[:5]:
  print(w, conditional_probs[w])


Top 5
and 0.4694915254237288
of 0.2896
in 0.2870508474576271
to 0.2600406779661017
experience 0.19086101694915253
Bottom 5
crosslly 1.3559322033898305e-05
Nebraska 1.3559322033898305e-05
aeroespacial 1.3559322033898305e-05
automotriz 1.3559322033898305e-05
-Typhoon 1.3559322033898305e-05
Top 5 Conditional probability (Responsibility)
years_Experience 0.8038114122053316
experience_Experience 0.7933465521039876
and_Responsibility 0.677983876253523
of_Experience 0.6488213262833223
and_SoftSkill 0.54428632115548


In [ ]:
# Calculate accuracy using dev dataset




In [ ]:
# Calculate accuracy using dev dataset

# y_hat = NBC(sentence)
# y_hat ==? y



## References

CSV reader example code:\
https://realpython.com/python-csv/

https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

https://www.geeksforgeeks.org/python-stemming-words-with-nltk/?ref=lbp

https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe

https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value